In [1]:
import twitter
import json

# XXX: Go to http://dev.twitter.com/apps/new to create an app and get values
# for these credentials, which you'll need to provide in place of these
# empty string values that are defined as placeholders.
# See https://dev.twitter.com/docs/auth/oauth for more information 
# on Twitter's OAuth implementation.

CONSUMER_KEY = "xxxxxxxxx"
CONSUMER_SECRET ="xxxxxxxx"
OAUTH_TOKEN = "xxxxxxxx"
OAUTH_TOKEN_SECRET = "xxxxxxxxx"

auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                           CONSUMER_KEY, CONSUMER_SECRET)

twitter_api = twitter.Twitter(auth=auth)

# Nothing to see by displaying twitter_api except that it's now a
# defined variable

print (twitter_api)

In [2]:
# Import unquote to prevent url encoding errors in next_results
from urllib.parse import unquote

# XXX: Set this variable to a trending topic, 
# or anything else for that matter. The example query below
# was a trending topic when this content was being developed
# and is used throughout the remainder of this chapter.
#q = '#XXX'
q = 'kitchen aid mixer'

count = 100

# See https://dev.twitter.com/docs/api/1.1/get/search/tweets

search_results = twitter_api.search.tweets(q=q, count=count)

statuses = search_results['statuses']


# Iterate through 5 more batches of results by following the cursor

for _ in range(5):
    print ("Length of statuses", len(statuses))
    try:
        next_results = search_results['search_metadata']['next_results']
    except KeyError as e: # No more results when next_results doesn't exist
        break
        
    # Create a dictionary from next_results, which has the following form:
    # ?max_id=313519052523986943&q=NCAA&include_entities=1
    kwargs = dict([ kv.split('=') for kv in unquote(next_results[1:]).split("&") ])
    
    search_results = twitter_api.search.tweets(**kwargs)
    statuses += search_results['statuses']

# Show one sample search result by slicing the list...
#json.dumps(statuses[0], indent=1)

print(json.dumps(statuses[0], indent=1))

Length of statuses 100
Length of statuses 196
Length of statuses 196
{
 "created_at": "Thu Dec 19 14:20:48 +0000 2019",
 "id": 1207667097427419143,
 "id_str": "1207667097427419143",
 "text": "I just use my Kitchen Aid mixer for the first time and I am in LOVE",
 "truncated": false,
 "entities": {
  "hashtags": [],
  "symbols": [],
  "user_mentions": [],
  "urls": []
 },
 "metadata": {
  "iso_language_code": "en",
  "result_type": "recent"
 },
 "source": "<a href=\"http://twitter.com/download/android\" rel=\"nofollow\">Twitter for Android</a>",
 "in_reply_to_status_id": null,
 "in_reply_to_status_id_str": null,
 "in_reply_to_user_id": null,
 "in_reply_to_user_id_str": null,
 "in_reply_to_screen_name": null,
 "user": {
  "id": 410136859,
  "id_str": "410136859",
  "name": "Tori",
  "screen_name": "_torrrrrnado",
  "location": "closer to hell than u",
  "description": "doin me and im doin it well",
  "url": null,
  "entities": {
   "description": {
    "urls": []
   }
  },
  "protected": 

In [3]:
status_texts = [ status['text'] 
                 for status in statuses ]

screen_names = [ user_mention['screen_name'] 
                 for status in statuses
                     for user_mention in status['entities']['user_mentions'] ]

hashtags = [ hashtag['text'] 
             for status in statuses
                 for hashtag in status['entities']['hashtags'] ]

# Compute a collection of all words from all tweets
words = [ w 
          for t in status_texts 
              for w in t.split() ]

# Explore the first 5 items for each...

print (json.dumps(status_texts[0:5], indent=1))
print (json.dumps(screen_names[0:5], indent=1))
print (json.dumps(hashtags[0:5], indent=1))
print (json.dumps(words[0:5], indent=1))

[
 "I just use my Kitchen Aid mixer for the first time and I am in LOVE",
 "When they said don\u2019t operate heavy machinery while drunk I think they meant my kitchen aid stand mixer",
 "LUCAS GOT ME A KITCHEN AID MIXER FOR CHRISTMAS IM SO HAPPY I COULD CRY !!! https://t.co/NXr7QY4snv",
 "Might fuck around and buys a Kitchen Aid standing mixer this Boxing Day \ud83d\udc40 Who wants to bake cinnamon buns and cookies?",
 "The wife already wrapped our families new Kitchen Aid mixer.\nCan't open it until Christmas!\nShe won't tell me what\u2026 https://t.co/ssajIvUVrd"
]
[
 "ColourPopCo",
 "MrsSarahMonster",
 "Postmates",
 "OMGitsagirl2015",
 "RobinsoMartina"
]
[
 "colourpopgivesback",
 "BetterThanSanta",
 "ArmorAll",
 "Repost",
 "AllTheMeringue"
]
[
 "I",
 "just",
 "use",
 "my",
 "Kitchen"
]


In [4]:
#change to lower cases to ignore sensitivity of upper and lower case
words = [word.lower() for word in words]
screen_names = [word.lower() for word in screen_names]
hashtags = [word.lower() for word in hashtags]

#remove 'RT' from word list
rt = ['rt']
for word in list(words):
    if word in rt:
        words.remove(word)

#new list of words without RT
words = [word for word in words if word not in rt]

from collections import Counter

for item in [words, screen_names, hashtags]:
    c = Counter(item)
    print (c.most_common()[:10]) # top 10

[('kitchen', 176), ('aid', 171), ('a', 153), ('i', 133), ('mixer', 119), ('my', 84), ('to', 84), ('and', 59), ('for', 44), ('the', 42)]
[('laurenthehough', 4), ('victoriabwrites', 3), ('robinsomartina', 2), ('nottinghilldady', 2), ('dorkusrob', 2), ('pickinguptoys', 2), ('reallylolo', 2), ('tomthomas578', 2), ('daddypoppinsblg', 2), ('aglayalilich', 2)]
[('colourpopgivesback', 1), ('betterthansanta', 1), ('armorall', 1), ('repost', 1), ('allthemeringue', 1), ('kitchenaid', 1), ('dreamsdocometrue', 1), ('canfitpro', 1), ('canfitprowrapped', 1), ('humankinetics', 1)]


In [5]:
from prettytable import PrettyTable

for label, data in (('Word', words), 
                    ('Screen Name', screen_names), 
                    ('Hashtag', hashtags)):
    pt = PrettyTable(field_names=[label, 'Count']) 
    c = Counter(data)
    [ pt.add_row(kv) for kv in c.most_common()[:10] ]
    pt.align[label], pt.align['Count'] = 'l', 'r' # Set column alignment
    print (pt)

+---------+-------+
| Word    | Count |
+---------+-------+
| kitchen |   176 |
| aid     |   171 |
| a       |   153 |
| i       |   133 |
| mixer   |   119 |
| my      |    84 |
| to      |    84 |
| and     |    59 |
| for     |    44 |
| the     |    42 |
+---------+-------+
+-----------------+-------+
| Screen Name     | Count |
+-----------------+-------+
| laurenthehough  |     4 |
| victoriabwrites |     3 |
| robinsomartina  |     2 |
| nottinghilldady |     2 |
| dorkusrob       |     2 |
| pickinguptoys   |     2 |
| reallylolo      |     2 |
| tomthomas578    |     2 |
| daddypoppinsblg |     2 |
| aglayalilich    |     2 |
+-----------------+-------+
+--------------------+-------+
| Hashtag            | Count |
+--------------------+-------+
| colourpopgivesback |     1 |
| betterthansanta    |     1 |
| armorall           |     1 |
| repost             |     1 |
| allthemeringue     |     1 |
| kitchenaid         |     1 |
| dreamsdocometrue   |     1 |
| canfitpro       

In [6]:
# A function for computing lexical diversity
def lexical_diversity(tokens):
    return 1.0*len(set(tokens))/len(tokens) 

# A function for computing the average number of words per tweet
def average_words(statuses):
    total_words = sum([ len(s.split()) for s in statuses ]) 
    return 1.0*total_words/len(statuses)

print (lexical_diversity(words))
print (lexical_diversity(screen_names))
print (lexical_diversity(hashtags))
print (average_words(status_texts))

0.3458026509572901
0.7916666666666666
1.0
17.397959183673468
